In [1]:
library('SelectSim')
library('tidyverse')
library('ggplot2')
library('ggpubr')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] ggpubr_0.6.0      lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

In [3]:
`%notin%` <- Negate(`%in%`)

# Note
- Ensure the file path are correct in read functions below of the raw downloaded data.
- Ensure the file are saved to correct location based on path decided by the user.

# Metadata 

In [4]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples<-readRDS(file='../data/processed/genie/msk_one_patient_one_sample_final.rds')
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples<-readRDS(file='../data/processed/genie/dfci_one_patient_one_sample_final.rds')

# MAF

In [5]:
genie_maf <- read.delim('../data/raw/genie/data_mutations_extended.txt')

# MSK GAMs Generation

In [6]:
genie_maf_msk <- genie_maf %>% select(Chromosome,Start_Position,End_Position,Hugo_Symbol,Variant_Classification,Tumor_Sample_Barcode,HGVSp_Short) %>%
filter(Tumor_Sample_Barcode %in% genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples$SAMPLE_ID)

In [7]:
msk_genes<-oncokb_genes

In [8]:
genie_maf_msk$sample<-genie_maf_msk$Tumor_Sample_Barcode

In [9]:
input_maf <- genie_maf_msk
print(paste('##### Number of lines ####',nrow(input_maf),sep="->"))
genes_to_consider =  msk_genes
print(paste('##### Number of genes ####',length(genes_to_consider),sep="->"))

[1] "##### Number of lines ####->413413"
[1] "##### Number of genes ####->396"


In [10]:
mutation_type = list(
      'ignore' = c("Silent","Intron","RNA","3'UTR","5'UTR","5'Flank","3'Flank","IGR"),
      'truncating'= c('Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins','Nonsense_Mutation','Nonstop_Mutation','Splice_Region','Splice_Site','Translation_Start_Site'),
      'missense' = c('Missense_Mutation')
)
custom_maf_schema = list(
    'name' = 'custom_maf',
    'column' = list(
          'gene' = 'Hugo_Symbol'
        , 'gene.name' = 'Hugo_Symbol'
        , 'sample' = 'sample'
        , 'sample.name' = 'sample'
        , 'mutation.type' = 'Variant_Classification'
        , 'mutation' = 'HGVSp_Short'
        ),
        'mutation.type' = mutation_type
)

In [11]:
head(genie_maf_msk)

,Chromosome,Start_Position,End_Position,Hugo_Symbol,Variant_Classification,Tumor_Sample_Barcode,HGVSp_Short,sample
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,118165602,118165602,FAM46C,Nonsense_Mutation,GENIE-MSK-P-0081662-T01-IM7,p.R38*,GENIE-MSK-P-0081662-T01-IM7
2,9,27209136,27209136,TEK,Missense_Mutation,GENIE-MSK-P-0039091-T01-IM6,p.D865Y,GENIE-MSK-P-0039091-T01-IM6
3,20,31372564,31372564,DNMT3B,Missense_Mutation,GENIE-MSK-P-0013676-T01-IM5,p.D69Y,GENIE-MSK-P-0013676-T01-IM5
4,4,143003288,143003288,INPP4B,Missense_Mutation,GENIE-MSK-P-0043596-T01-IM6,p.K846N,GENIE-MSK-P-0043596-T01-IM6
5,17,70118924,70118926,SOX9,In_Frame_Del,GENIE-MSK-P-0076002-T01-IM7,p.K167del,GENIE-MSK-P-0076002-T01-IM7
6,16,347904,347904,AXIN1,Missense_Mutation,GENIE-MSK-P-0002914-T01-IM3,p.H534Q,GENIE-MSK-P-0002914-T01-IM3


In [12]:
mut_samples = unique(input_maf[, custom_maf_schema$column$sample])
print(paste('##### Number of samples ####',length(mut_samples),sep="->"))

[1] "##### Number of samples ####->42790"


In [13]:
maf_genes = filter_maf_gene.name(input_maf, genes = genes_to_consider, gene.col = custom_maf_schema$column$gene)
print(paste('##### Number of lines ####',nrow(maf_genes),sep="->"))

[1] "##### Number of lines ####->336381"


In [14]:
head(input_maf)

,Chromosome,Start_Position,End_Position,Hugo_Symbol,Variant_Classification,Tumor_Sample_Barcode,HGVSp_Short,sample
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,118165602,118165602,FAM46C,Nonsense_Mutation,GENIE-MSK-P-0081662-T01-IM7,p.R38*,GENIE-MSK-P-0081662-T01-IM7
2,9,27209136,27209136,TEK,Missense_Mutation,GENIE-MSK-P-0039091-T01-IM6,p.D865Y,GENIE-MSK-P-0039091-T01-IM6
3,20,31372564,31372564,DNMT3B,Missense_Mutation,GENIE-MSK-P-0013676-T01-IM5,p.D69Y,GENIE-MSK-P-0013676-T01-IM5
4,4,143003288,143003288,INPP4B,Missense_Mutation,GENIE-MSK-P-0043596-T01-IM6,p.K846N,GENIE-MSK-P-0043596-T01-IM6
5,17,70118924,70118926,SOX9,In_Frame_Del,GENIE-MSK-P-0076002-T01-IM7,p.K167del,GENIE-MSK-P-0076002-T01-IM7
6,16,347904,347904,AXIN1,Missense_Mutation,GENIE-MSK-P-0002914-T01-IM3,p.H534Q,GENIE-MSK-P-0002914-T01-IM3


In [15]:
genie_maf_msk %>% count(Variant_Classification)

Variant_Classification,n
<chr>,<int>
3'Flank,878
3'UTR,122
5'Flank,4774
5'UTR,253
Frame_Shift_Del,39357
Frame_Shift_Ins,14971
In_Frame_Del,6858
In_Frame_Ins,1483
Intron,725


In [16]:
tictoc::tic('##### Creating Truncating GAM ####')
    maf_trunc = filter_maf_truncating(maf_genes,genes=oncokb_truncating_genes, custom_maf_schema)
    input_maf_trunc<-filter_maf_truncating(input_maf, custom_maf_schema)
    truncating_tmb <- data.frame('sample'=mut_samples,'mutation'=rep(0,length(mut_samples)))
    rownames(truncating_tmb)<-mut_samples
    temp <- input_maf_trunc %>% count(sample) 
    rownames(temp)<-temp$sample
    truncating_tmb[intersect(truncating_tmb$sample,temp$sample),]$mutation <-temp[intersect(truncating_tmb$sample,temp$sample),'n']
    tcga_truc_gam = maf2gam(maf_trunc,
                     sample.col = custom_maf_schema$column$sample,
                     gene.col = custom_maf_schema$column$gene,
                     value.var = 'Variant_Classification',
                     samples = mut_samples,
                     genes = genes_to_consider,
                     fun.aggregate = length,
                     binarize=TRUE,
                     fill=0)
    truncating_data <- list('gam'=tcga_truc_gam,
                            'tmb'=truncating_tmb)
tictoc::toc()

##### Creating Truncating GAM ####: 15.95 sec elapsed


In [17]:
tictoc::tic('##### Creating Missense GAM ####')
    maf_valid = filter_maf_schema(input_maf,
                             schema = custom_maf_schema,
                             column = 'mutation.type',
                             values = custom_maf_schema[['mutation.type']][['ignore']],
                             inclusive = FALSE)
    missense_maf<-filter_maf_mutation.type(input_maf,
                                      variants = 'Missense_Mutation',
                                      variant.col = custom_maf_schema$column$mutation.type)
    missense_tmb <- data.frame('sample'=mut_samples,'mutation'=rep(0,length(mut_samples)))
    rownames(missense_tmb)<-mut_samples
    temp <- missense_maf %>% count(sample) 
    rownames(temp)<-temp$sample
    missense_tmb[intersect(missense_tmb$sample,temp$sample),]$mutation <-temp[intersect(missense_tmb$sample,temp$sample),'n']
    t_m = substr(maf_valid[[custom_maf_schema$column$mutation]],3,1000)
    t_m1 =  gsub('[A-Z]*$', '', t_m)
    maf_valid$HGVSp_Short_fixed = t_m1
    maf_hotspot = filter_maf_mutations(maf_valid,
                                  variant_catalogue,
                                  maf.col = c(custom_maf_schema$column$gene, 'HGVSp_Short_fixed'),
                                  values.col = c('gene', 'mut'))

    missense_tcga_gam = maf2gam(maf_hotspot,
                     sample.col = custom_maf_schema$column$sample,
                     gene.col = custom_maf_schema$column$gene,
                     value.var = 'Variant_Classification',
                     samples = mut_samples,
                     genes = genes_to_consider,
                     fun.aggregate = length,
                     binarize=TRUE,
                     fill=0)
    missesne_data <- list('gam'=missense_tcga_gam,
                          'tmb'=missense_tmb)

tictoc::toc()

##### Creating Missense GAM ####: 12.068 sec elapsed


In [18]:
str(truncating_data)
str(missesne_data)

List of 2
 $ gam: num [1:42790, 1:396] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:42790] "GENIE-MSK-P-0081662-T01-IM7" "GENIE-MSK-P-0039091-T01-IM6" "GENIE-MSK-P-0013676-T01-IM5" "GENIE-MSK-P-0043596-T01-IM6" ...
  .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
 $ tmb:'data.frame':	42790 obs. of  2 variables:
  ..$ sample  : chr [1:42790] "GENIE-MSK-P-0081662-T01-IM7" "GENIE-MSK-P-0039091-T01-IM6" "GENIE-MSK-P-0013676-T01-IM5" "GENIE-MSK-P-0043596-T01-IM6" ...
  ..$ mutation: num [1:42790] 45 13 42 87 2 0 1 7 31 0 ...
List of 2
 $ gam: num [1:42790, 1:396] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:42790] "GENIE-MSK-P-0081662-T01-IM7" "GENIE-MSK-P-0039091-T01-IM6" "GENIE-MSK-P-0013676-T01-IM5" "GENIE-MSK-P-0043596-T01-IM6" ...
  .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
 $ tmb:'data.frame':	42790 obs. of  2 variables:
  ..$ sample  : chr [1:42790] "GENIE-MSK-P-0081662-T01-IM7" "GENIE-MSK-P-0039091-T01-IM6" "

In [19]:
dim(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples)

[1] 42790    22

In [20]:
head(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples,2)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,⋯,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,class,Tumor_run_group,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-MSK-P-0000223,Female,White,Non-Spanish/non-Hispanic,MSK,11142,11142,2016,TRUE,2016,⋯,AASTR,Primary,MSK-IMPACT341,Glioma,Anaplastic Astrocytoma,Primary tumor,AASTR,AASTR,Glioma_low_grade,Glioma
2,GENIE-MSK-P-0000679,Female,White,Non-Spanish/non-Hispanic,MSK,22267,22267,2015,TRUE,2015,⋯,AASTR,Primary,MSK-IMPACT341,Glioma,Anaplastic Astrocytoma,Primary tumor,AASTR,AASTR,Glioma_low_grade,Glioma


In [21]:
sample_annoation <- (genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_ID %in% mut_samples))$final_class
names(sample_annoation)<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_ID %in% mut_samples))$SAMPLE_ID

In [22]:
str(sample_annoation)

 Named chr [1:42790] "Glioma_low_grade" "Glioma_low_grade" ...
 - attr(*, "names")= chr [1:42790] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...


## Creating Primary Run GAM

In [23]:
primary_samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE=='Primary'))$SAMPLE_ID

In [24]:
gene_to_take <- colnames(missesne_data$gam)
order <- primary_samples

data <-list('M'=list('missense'=t(missesne_data$gam[order,gene_to_take]),
                     'truncating'=t(truncating_data$gam[rownames(missesne_data$gam[order,]),gene_to_take])),
            'tmb'=list('missense'=missesne_data$tmb[order,],
                       'truncating'=truncating_data$tmb[order,]))

alteration_covariates <- rep('MUT',ncol(missesne_data$gam[order,gene_to_take]))
names(alteration_covariates)<-colnames(missesne_data$gam[order,gene_to_take])

In [25]:
run_data <- list('M'=data,'sample.class' = sample_annoation[order],'alteration.class' = alteration_covariates)
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. ..$ truncating: num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	27825 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. .. ..$ mutation: num [1:27825] 4 4 1 0 4 2 3 3 2 3 ..

In [26]:
saveRDS(run_data,file='../data/processed/gams/pan_can_msk_primary_run_data_v15.rds')

## Creating Metastasis Run GAM

In [27]:
meta_samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE!='Primary'))$SAMPLE_ID

In [28]:
gene_to_take <- colnames(missesne_data$gam)
order <- meta_samples

data <-list('M'=list('missense'=t(missesne_data$gam[order,gene_to_take]),
                     'truncating'=t(truncating_data$gam[rownames(missesne_data$gam[order,]),gene_to_take])),
            'tmb'=list('missense'=missesne_data$tmb[order,],
                       'truncating'=truncating_data$tmb[order,]))

alteration_covariates <- rep('MUT',ncol(missesne_data$gam[order,gene_to_take]))
names(alteration_covariates)<-colnames(missesne_data$gam[order,gene_to_take])

In [29]:
run_data <- list('M'=data,'sample.class' = sample_annoation[order],'alteration.class' = alteration_covariates)
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:14965] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14965] "GENIE-MSK-P-0020921-T01-IM6" "GENIE-MSK-P-0027135-T01-IM6" "GENIE-MSK-P-0045988-T02-IM6" "GENIE-MSK-P-0001029-T01-IM3" ...
  .. ..$ truncating: num [1:396, 1:14965] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14965] "GENIE-MSK-P-0020921-T01-IM6" "GENIE-MSK-P-0027135-T01-IM6" "GENIE-MSK-P-0045988-T02-IM6" "GENIE-MSK-P-0001029-T01-IM3" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	14965 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:14965] "GENIE-MSK-P-0020921-T01-IM6" "GENIE-MSK-P-0027135-T01-IM6" "GENIE-MSK-P-0045988-T02-IM6" "GENIE-MSK-P-0001029-T01-IM3" ...
  .. .. ..$ mutation: num [1:14965] 7 3 1 1 1 3 2 6 1 1 ..

In [30]:
saveRDS(run_data,file='../data/processed/gams/pan_can_msk_meta_run_data_v15.rds')

# DFCI GAMs Generation

In [31]:
genie_maf_dfci <- genie_maf %>% select(Chromosome,Start_Position,End_Position,Hugo_Symbol,Variant_Classification,Tumor_Sample_Barcode,HGVSp_Short) %>%
filter(Tumor_Sample_Barcode %in% genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$SAMPLE_ID)

In [32]:
dfci_genes<-oncokb_genes

In [33]:
genie_maf_dfci$sample<-genie_maf_dfci$Tumor_Sample_Barcode

In [34]:
input_maf <- genie_maf_dfci
print(paste('##### Number of lines ####',nrow(input_maf),sep="->"))
genes_to_consider =  dfci_genes
print(paste('##### Number of genes ####',length(genes_to_consider),sep="->"))

[1] "##### Number of lines ####->284213"
[1] "##### Number of genes ####->396"


In [35]:
mutation_type = list(
      'ignore' = c("Silent","Intron","RNA","3'UTR","5'UTR","5'Flank","3'Flank","IGR"),
      'truncating'= c('Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins','Nonsense_Mutation','Nonstop_Mutation','Splice_Region','Splice_Site','Translation_Start_Site'),
      'missense' = c('Missense_Mutation')
)
custom_maf_schema = list(
    'name' = 'custom_maf',
    'column' = list(
          'gene' = 'Hugo_Symbol'
        , 'gene.name' = 'Hugo_Symbol'
        , 'sample' = 'sample'
        , 'sample.name' = 'sample'
        , 'mutation.type' = 'Variant_Classification'
        , 'mutation' = 'HGVSp_Short'
        ),
        'mutation.type' = mutation_type
)

In [36]:
head(genie_maf_dfci)

,Chromosome,Start_Position,End_Position,Hugo_Symbol,Variant_Classification,Tumor_Sample_Barcode,HGVSp_Short,sample
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,19,42777065,42777065,CIC,Missense_Mutation,GENIE-DFCI-183676-2518535,p.K377R,GENIE-DFCI-183676-2518535
2,9,139399555,139399555,NOTCH1,Missense_Mutation,GENIE-DFCI-183676-2518535,p.P1530S,GENIE-DFCI-183676-2518535
3,11,108163428,108163431,ATM,Frame_Shift_Del,GENIE-DFCI-183676-2518535,p.Y1508Vfs*5,GENIE-DFCI-183676-2518535
4,8,41790280,41790280,KAT6A,Missense_Mutation,GENIE-DFCI-183676-2518535,p.T1820A,GENIE-DFCI-183676-2518535
5,7,55233012,55233012,EGFR,Missense_Mutation,GENIE-DFCI-183676-2518535,p.G588S,GENIE-DFCI-183676-2518535
6,11,94192612,94192612,MRE11A,Missense_Mutation,GENIE-DFCI-183676-2518535,p.R488C,GENIE-DFCI-183676-2518535


In [37]:
mut_samples = unique(input_maf[, custom_maf_schema$column$sample])
print(paste('##### Number of samples ####',length(mut_samples),sep="->"))

[1] "##### Number of samples ####->22264"


In [38]:
maf_genes = filter_maf_gene.name(input_maf, genes = genes_to_consider, gene.col = custom_maf_schema$column$gene)
print(paste('##### Number of lines ####',nrow(maf_genes),sep="->"))

[1] "##### Number of lines ####->192526"


In [39]:
tictoc::tic('##### Creating Truncating GAM ####')
    maf_trunc = filter_maf_truncating(maf_genes,genes=oncokb_truncating_genes, custom_maf_schema)
    input_maf_trunc<-filter_maf_truncating(input_maf, custom_maf_schema)
    truncating_tmb <- data.frame('sample'=mut_samples,'mutation'=rep(0,length(mut_samples)))
    rownames(truncating_tmb)<-mut_samples
    temp <- input_maf_trunc %>% count(sample) 
    rownames(temp)<-temp$sample
    truncating_tmb[intersect(truncating_tmb$sample,temp$sample),]$mutation <-temp[intersect(truncating_tmb$sample,temp$sample),'n']
    tcga_truc_gam = maf2gam(maf_trunc,
                     sample.col = custom_maf_schema$column$sample,
                     gene.col = custom_maf_schema$column$gene,
                     value.var = 'Variant_Classification',
                     samples = mut_samples,
                     genes = genes_to_consider,
                     fun.aggregate = length,
                     binarize=TRUE,
                     fill=0)
    truncating_data <- list('gam'=tcga_truc_gam,
                            'tmb'=truncating_tmb)
tictoc::toc()

##### Creating Truncating GAM ####: 4.393 sec elapsed


In [40]:
tictoc::tic('##### Creating Missense GAM ####')
    maf_valid = filter_maf_schema(input_maf,
                             schema = custom_maf_schema,
                             column = 'mutation.type',
                             values = custom_maf_schema[['mutation.type']][['ignore']],
                             inclusive = FALSE)
    missense_maf<-filter_maf_mutation.type(input_maf,
                                      variants = 'Missense_Mutation',
                                      variant.col = custom_maf_schema$column$mutation.type)
    missense_tmb <- data.frame('sample'=mut_samples,'mutation'=rep(0,length(mut_samples)))
    rownames(missense_tmb)<-mut_samples
    temp <- missense_maf %>% count(sample) 
    rownames(temp)<-temp$sample
    missense_tmb[intersect(missense_tmb$sample,temp$sample),]$mutation <-temp[intersect(missense_tmb$sample,temp$sample),'n']
    t_m = substr(maf_valid[[custom_maf_schema$column$mutation]],3,1000)
    t_m1 =  gsub('[A-Z]*$', '', t_m)
    maf_valid$HGVSp_Short_fixed = t_m1
    maf_hotspot = filter_maf_mutations(maf_valid,
                                  variant_catalogue,
                                  maf.col = c(custom_maf_schema$column$gene, 'HGVSp_Short_fixed'),
                                  values.col = c('gene', 'mut'))

    missense_tcga_gam = maf2gam(maf_hotspot,
                     sample.col = custom_maf_schema$column$sample,
                     gene.col = custom_maf_schema$column$gene,
                     value.var = 'Variant_Classification',
                     samples = mut_samples,
                     genes = genes_to_consider,
                     fun.aggregate = length,
                     binarize=TRUE,
                     fill=0)
    missesne_data <- list('gam'=missense_tcga_gam,
                          'tmb'=missense_tmb)

tictoc::toc()

##### Creating Missense GAM ####: 5.791 sec elapsed


In [41]:
str(truncating_data)
str(missesne_data)

List of 2
 $ gam: num [1:22264, 1:396] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:22264] "GENIE-DFCI-183676-2518535" "GENIE-DFCI-000605-5717" "GENIE-DFCI-006285-6798" "GENIE-DFCI-008427-5883" ...
  .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
 $ tmb:'data.frame':	22264 obs. of  2 variables:
  ..$ sample  : chr [1:22264] "GENIE-DFCI-183676-2518535" "GENIE-DFCI-000605-5717" "GENIE-DFCI-008456-8063" "GENIE-DFCI-006285-6798" ...
  ..$ mutation: num [1:22264] 4 2 1 1 2 1 3 1 0 3 ...
List of 2
 $ gam: num [1:22264, 1:396] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:22264] "GENIE-DFCI-183676-2518535" "GENIE-DFCI-000605-5717" "GENIE-DFCI-008456-8063" "GENIE-DFCI-008516-7383" ...
  .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
 $ tmb:'data.frame':	22264 obs. of  2 variables:
  ..$ sample  : chr [1:22264] "GENIE-DFCI-183676-2518535" "GENIE-DFCI-000605-5717" "GENIE-DFCI-008456-8063" "GENIE-DFCI-006285-6798" ...
  ..$ muta

In [42]:
dim(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples)

[1] 22264    20

In [43]:
head(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples,2)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer
2,GENIE-DFCI-007383,Male,White,Non-Spanish/non-Hispanic,DFCI,23668,23673,2015,True,2015,GENIE-DFCI-007383-3930,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [44]:
sample_annoation <- (genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_ID %in% mut_samples))$final_class
names(sample_annoation)<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_ID %in% mut_samples))$SAMPLE_ID

In [45]:
str(sample_annoation)

 Named chr [1:22264] "LUAD" "LUAD" "BLCA" "LUAD" "LUSC" "THPA" "STAD" "IDC" ...
 - attr(*, "names")= chr [1:22264] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000018-4015" "GENIE-DFCI-000544-5758" ...


## Creating Primary Run GAM

In [46]:
primary_samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE=='Primary'))$SAMPLE_ID

In [47]:
gene_to_take <- colnames(missesne_data$gam)
order <- primary_samples

data <-list('M'=list('missense'=t(missesne_data$gam[order,gene_to_take]),
                     'truncating'=t(truncating_data$gam[rownames(missesne_data$gam[order,]),gene_to_take])),
            'tmb'=list('missense'=missesne_data$tmb[order,],
                       'truncating'=truncating_data$tmb[order,]))

alteration_covariates <- rep('MUT',ncol(missesne_data$gam[order,gene_to_take]))
names(alteration_covariates)<-colnames(missesne_data$gam[order,gene_to_take])

In [48]:
run_data <- list('M'=data,'sample.class' = sample_annoation[order],'alteration.class' = alteration_covariates)
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:14657] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  .. ..$ truncating: num [1:396, 1:14657] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	14657 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  .. .. ..$ mutation: num [1:14657] 6 9 3 5 1 1 7 5 0 2 ...
  .. ..$ truncating:'data.frame':	14657 obs. of  2 variabl

In [49]:
saveRDS(run_data,file='../data/processed/gams/pan_can_dfci_primary_run_data_v15.rds')

## Creating Metastasis Run GAM

In [50]:
meta_samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE!='Primary'))$SAMPLE_ID

In [51]:
gene_to_take <- colnames(missesne_data$gam)
order <- meta_samples

data <-list('M'=list('missense'=t(missesne_data$gam[order,gene_to_take]),
                     'truncating'=t(truncating_data$gam[rownames(missesne_data$gam[order,]),gene_to_take])),
            'tmb'=list('missense'=missesne_data$tmb[order,],
                       'truncating'=truncating_data$tmb[order,]))

alteration_covariates <- rep('MUT',ncol(missesne_data$gam[order,gene_to_take]))
names(alteration_covariates)<-colnames(missesne_data$gam[order,gene_to_take])

In [52]:
run_data <- list('M'=data,'sample.class' = sample_annoation[order],'alteration.class' = alteration_covariates)
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:7607] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  .. ..$ truncating: num [1:396, 1:7607] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	7607 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  .. .. ..$ mutation: num [1:7607] 2 5 11 1 2 4 4 5 4 5 ...
  .. ..$ truncating:'data.frame':	7607 obs. of  2 variables:
  .

In [53]:
saveRDS(run_data,file='../data/processed/gams/pan_can_dfci_meta_run_data_v15.rds')